In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# &#128013; [One Point Tutorial] Visualization - Practice
<p style='text-align: right;'> Python을 활용한 데이터 시각화 </p>
<p style='text-align: right;'> December, 2019</p>

---

## *<font color="green">실제 데이터(제주도 버스 승하차 데이터)</font>*를 기반으로 실습을 해보자!


### 학습 목표
`다양한 형태의 데이터`에 대한 핵심적인 시각화 기법을 이해하고 실습함

### 목차
 1. folium Basic
 2. folium Advanced

 

### <font color="royalblue">첫번째</font>: Import module
- 데이터를 불러오고 시각화를 시키기 위해 필요한 모듈을 불러옵니다!

In [2]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import folium

import os

In [3]:
try:
    os.chdir("D:\Portfolio\[project]daycon-제주버스정류장 승하차 예측")
    print("Current Working Directory is changed.")
except OSError:
    print("Can't change Current Working Directory.") 

Current Working Directory is changed.


다음 친구는 경고 메세지를 안 뜨게 해주는 모듈인데 보기 편하도록 불러오도록 하겠습니다.<br>
(안 불러와도 크게 지장은 없는 친구입니다!)

In [4]:
import warnings
warnings.filterwarnings('ignore')

다음으로, 그래프 테마를 적용하겠습니다.

In [5]:
plt.style.use('ggplot')

### <font color="royalblue">두번째</font>: import data (데이터 불러오기)

- 먼저 데이터부터 다운로드 받고, 각자의 작업폴더로 파일을 이동합니다.

In [6]:
data = pd.read_csv("train.csv", encoding="utf8") #encoding 방법은 개인 컴퓨터마다 다를 수 있습니다 !
data.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,...,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,...,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,...,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,0.0,17.0,...,26.0,14.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0
4,4,2019-09-01,4270000,시내,1646,중문관광단지입구,33.25579,126.41260,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


**<train.csv dataset 설명>**  
2019년 9월 제주도의 각 날짜, 출근시간(6시~12시)의 버스 정류장별 승하차 인원, 퇴근시간(18시~20시)의 버스 정류장별 승차 인원 기록

- 제대로 된 시각화를 위해서 변수에 대한 이해는 필수적이며, 역으로 변수에 대한 이해를 위해서도 시각화는 필수적입니다.

#### <div style=text-align:center> [ Table Description ] </div>
|변수명|설명|
|:---:|:---:|
|id|해당 데이터에서의 고유한 ID|
|date|날짜|
|bus_route_id|버스 노선 ID|
|in_out|시내버스, 시외버스 구분|
|station_code|해당 승하차 정류소의 ID|
|station_name|해당 승하차 정류소의 이름|
|latitude|해당 버스 정류장의 위도 (같은 정류장 이름이어도 버스의 진행 방향에 따라 다를 수 있음)|
|longtitude|해당 버스 정류장의 경도 (같은 정류장 이름이어도 버스의 진행 방향에 따라 다를 수 있음)|
|6~7_ride|6:00:00부터 6:59:59까지 승차한 인원 수|
|7~8_ride|7:00:00부터 7:59:59까지 승차한 인원 수|
|8~9_ride|8:00:00부터 8:59:59까지 승차한 인원 수|
|9~10_ride|9:00:00부터 9:59:59까지 승차한 인원 수|
|10~11_ride|10:00:00부터 10:59:59까지 승차한 인원 수|
|11~12_ride|11:00:00부터 11:59:59까지 승차한 인원 수|
|6~7_takeoff|6:00:00부터 6:59:59까지 하차한 인원 수|
|7~8_takeoff|7:00:00부터 7:59:59까지 하차한 인원 수|
|8~9_takeoff|8:00:00부터 8:59:59까지 하차한 인원 수|
|9~10_takeoff|9:00:00부터 9:59:59까지 하차한 인원 수|
|10~11_takeoff|10:00:00부터 10:59:59까지 하차한 인원 수|
|11~12_takeoff|11:00:00부터 11:59:59까지 하차한 인원 수|
|18~20_ride|18:00:00부터 19:59:59까지 승차한 인원 수|

### <font color="royalblue">세번째</font> : 데이터 살펴보기 (간단한 EDA)

#### 데이터 살펴보기

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415423 entries, 0 to 415422
Data columns (total 21 columns):
id               415423 non-null int64
date             415423 non-null object
bus_route_id     415423 non-null int64
in_out           415423 non-null object
station_code     415423 non-null int64
station_name     415423 non-null object
latitude         415423 non-null float64
longitude        415423 non-null float64
6~7_ride         415423 non-null float64
7~8_ride         415423 non-null float64
8~9_ride         415423 non-null float64
9~10_ride        415423 non-null float64
10~11_ride       415423 non-null float64
11~12_ride       415423 non-null float64
6~7_takeoff      415423 non-null float64
7~8_takeoff      415423 non-null float64
8~9_takeoff      415423 non-null float64
9~10_takeoff     415423 non-null float64
10~11_takeoff    415423 non-null float64
11~12_takeoff    415423 non-null float64
18~20_ride       415423 non-null float64
dtypes: float64(15), int64(3), object

- data는 데이터프레임의 형태를 가지고 있구나 !

In [8]:
data.shape

(415423, 21)

- 행은 총 415423개, 열은 총 21개가 있구나 !

In [9]:
data.describe()

,id,bus_route_id,station_code,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride
count,415423.000000,4.154230e+05,4.154230e+05,415423.000000,415423.000000,415423.000000,415423.000000,415423.000000,415423.000000,415423.000000,415423.000000,415423.000000,415423.000000,415423.000000,415423.000000,415423.000000,415423.000000,415423.000000
mean,207711.000000,2.691935e+07,4.209311e+04,33.420375,126.533574,0.305893,0.829699,0.815350,0.642475,0.599618,0.579393,0.112870,0.344870,0.516481,0.430922,0.408001,0.402874,1.242095
std,119922.434776,3.924652e+06,4.971504e+05,0.107996,0.140986,1.109766,2.255116,2.317561,1.959844,1.885941,1.942137,0.597714,1.279179,1.658850,1.485124,1.412839,1.446080,4.722287
min,0.000000,4.270000e+06,1.000000e+00,33.208350,126.165040,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,103855.500000,2.346000e+07,3.200000e+02,33.291080,126.475780,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,207711.000000,2.803000e+07,1.130000e+03,33.478850,126.529770,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,311566.500000,3.003000e+07,2.226000e+03,33.500020,126.575890,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,415422.000000,3.282000e+07,6.115101e+06,33.963640,126.965670,85.000000,94.000000,136.000000,78.000000,124.000000,99.000000,45.000000,66.000000,59.000000,65.000000,52.000000,81.000000,272.000000


- describe() 함수는 int 형태 변수들의 빈도, 평균, 편차, 최솟값, 25%, 50%, 75%, 최댓값을 보여준다 !

#### 데이터 변수 변환

#### 1. date 변수 변환


- date 변수를 datetime으로 변수형으로 변환해준다 !

In [10]:
data['date'] = pd.to_datetime(data['date'])

In [11]:
data['weekday'] = data['date'].dt.weekday #data에 새로운 변수 'weekday' 추가 -> 요일 정보

- weekday 변수의 값으로는 0~6까지의 값이 들어간다 !

#### 2. in_out 변수 변환

- 시내와 시외로 나누어져 있는 in_out 변수를 변환해주자 !

In [12]:
data['in_out'].value_counts() #value_counts() 함수는 변수값의 빈도를 세어준다 !

시내    408500
시외      6923
Name: in_out, dtype: int64

- 시내에 비해 시외가 값이 많이 작군 !
- 그렇다면, map 함수를 이용하여 'in_out' 변수를 binary 변수화 시켜보자 !

In [13]:
data['in_out'] = data['in_out'].map({'시내':0,'시외':1})

### <font color="royalblue">네번째</font> : 데이터 시각화

### <font color="hotpink">지도</font> 시각화

#### 지도 시각화에 이용할 데이터 확인

- 전체 데이터부터 확인해보자!

In [14]:
data.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,...,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride,weekday
0,0,2019-09-01,4270000,1,344,제주썬호텔,33.48990,126.49373,0.0,1.0,...,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
1,1,2019-09-01,4270000,1,357,한라병원,33.48944,126.48508,1.0,4.0,...,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,6
2,2,2019-09-01,4270000,1,432,정존마을,33.48181,126.47352,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6
3,3,2019-09-01,4270000,0,1579,제주국제공항(600번),33.50577,126.49252,0.0,17.0,...,14.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,6
4,4,2019-09-01,4270000,0,1646,중문관광단지입구,33.25579,126.41260,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6


### <font color="orange">지도 시각화 기본</font>_정류장의 위치를 나타내보자!

- 위도와 경도, 정류장 이름을 가져오자 !

In [15]:
addr2 = data[['station_name','latitude','longitude']] # 정류장 이름, 위도, 경도 뽑아내기
addr2 = addr2.drop_duplicates() # 위치 정보가 중복될 수 있으므로 중복 행 제거
addr2.head()

,station_name,latitude,longitude
0,제주썬호텔,33.48990,126.49373
1,한라병원,33.48944,126.48508
2,정존마을,33.48181,126.47352
3,제주국제공항(600번),33.50577,126.49252
4,중문관광단지입구,33.25579,126.41260


- 데이터 양이 너무 많으면 지도 시각화 하기가 힘들기 때문에 특정 위치 정보를 추출하자 !

In [16]:
addr2_front = addr2.sort_values(by=['latitude'], axis=0).head(10)
addr2_tail = addr2.sort_values(by=['latitude'], axis=0).tail(10)

- folium.Marker 함수를 이용하자 !

In [17]:
addr2_front[['latitude','longitude']].mean()

latitude      33.213454
longitude    126.268891
dtype: float64

In [18]:
addr2_tail[['latitude','longitude']].mean()

latitude      33.962469
longitude    126.295546
dtype: float64

#### addr2_front부터 이용하자 !
- addr2_front의 위/경도 평균으로 초기 지도 객체를 생성해주도록 하겠다 !

In [19]:
# 초기 지도 객체 생성
map_osm = folium.Map(location=[33.2134, 126.2688], zoom_start = 12)
map_osm

- 이제 정류장의 위치를 표시해보자 !
- 아이콘은 구름으로 설정해보자 !

In [20]:
for item in addr2_front.index:
    lat = addr2_front.loc[item, 'latitude']
    long = addr2_front.loc[item, 'longitude']
    folium.Marker([lat, long], #위도 경도
        popup = addr2_front.loc[item,'station_name'],
        icon=folium.Icon(icon='cloud')
    ).add_to(map_osm)
map_osm

#### addr2_tail도 이용하자 !

- addr2_tail의 위/경도 평균으로 초기 지도 객체를 생성해주도록 하겠다 !
- 테마도 한번 바꿔볼까?

In [21]:
# 초기 지도 객체 생성
map_osm = folium.Map(location=[33.9624, 126.2955], tiles='Stamen Toner', zoom_start = 15)
map_osm

- 이번에는 새로운 아이콘을 적용해보자 !

In [22]:
for item in addr2_tail.index:
    lat = addr2_tail.loc[item, 'latitude']
    long = addr2_tail.loc[item, 'longitude']
    folium.Marker([lat, long], #위도 경도
        popup = addr2_tail.loc[item,'station_name'],
        icon=folium.Icon(color='red',icon='info-sign')
    ).add_to(map_osm)
map_osm

- 색은 바꿀 수 있고, 아이콘은 i라고 뜬다 !
- tiles 옵션을 통해 지도가 보여지는 이미지도 바꿨다 !
- 각 아이콘 누르면 정류장 이름이 뜬다 !

### <font color="orange">지도 시각화 심화</font>_특정 값과 연계하여 위치 시각화를 해보자 !

- 지도 시각화를 하기 위해 필요한 데이터는 위도와 경도 !
- 전체 데이터에서 station_code (정류장 코드)와 latitude, longitude (위도/경도) 변수를 뽑아내자! 

In [23]:
addr = data[['station_code','latitude','longitude']] # 정류장 코드, 위도, 경도 뽑아내기
addr = addr.drop_duplicates() # 위치 정보가 중복될 수 있으므로 중복 행 제거
addr.head()

,station_code,latitude,longitude
0,344,33.48990,126.49373
1,357,33.48944,126.48508
2,432,33.48181,126.47352
3,1579,33.50577,126.49252
4,1646,33.25579,126.41260


In [24]:
ppl = data.groupby(['station_code'], as_index=False).sum() #station_name으로 그룹화하여 sum()을 통해 더해준 변수 ppl에 저장
ppl = ppl[['station_code','8~9_ride','8~9_takeoff']] #ppl에서 8~9시 승하차 데이터 뽑아낸다 !

ppl['ppl_sum'] = ppl['8~9_ride']+ppl['8~9_takeoff'] # 승하차 인원 더해준 값 ppl에 지정
ppl = ppl[['station_code','ppl_sum']] # 정류장 이름과 총 승하차 인원만 뽑아줌

ppl.head()

,station_code,ppl_sum
0,1,263.0
1,2,237.0
2,3,28.0
3,4,457.0
4,5,168.0


- 그렇다면 어떤 데이터를 시각화해볼까?
- 8~9 시간대의 각 정류장 별 승/하차 인원을 지도에 보여줘볼까?
- 그렇다면 8~9 시간대의 각 정류장 별 승/하차 인원의 정보를 알기 위해 데이터를 처리해야해!

- 이제 위도, 경도, 정류장 이름, 총 승하차 인원 정보가 담긴 데이터들을 합쳐주자 !
- pandas에 내장되어 있는 merge 함수를 이용해주면 된다 !

In [25]:
map_info = pd.merge(ppl, addr, how='inner') # inner join은 교집합 의미 !
map_info.head()

# 지도에 표시하기엔 데이터 양 많으니 station_code 상위 30개만 뽑아보자!
map_front = map_info.head(30)
map_front.head()

# 이번엔 하위!
map_tail = map_info.tail(30)
map_tail.head()

,station_code,ppl_sum,latitude,longitude
3533,6115019,4.0,33.94561,126.32767
3534,6115020,6.0,33.94603,126.32886
3535,6115023,0.0,33.95359,126.33134
3536,6115024,56.0,33.95134,126.32771
3537,6115025,0.0,33.94864,126.33099


- 우와! 정류장 코드와 총 승하차 인원, 위도/경도만 나타내는 데이터프레임이 만들어졌다 !

#### 본격적 지도 시각화 시작 !

- 가장 먼저 처음에 로딩할 지도 객체를 생성해준다 !
- folium.Map 함수 이용 !

- 초기 지도 객체 생성해줄 때 초기 위도, 경도 값 필요하므로 일단 가지고 있는 데이터의 위도와 경도 평균으로 해주자 !
- 아까 만들어놓은 map_info 사용 !

In [26]:
map_info[['latitude','longitude']].mean()

latitude      33.396730
longitude    126.532185
dtype: float64

In [27]:
# 초기 지도 객체 생성
map_osm = folium.Map(location=[33.3967, 126.5321], zoom_start = 11)
map_osm

- 오 ~ 제주도의 거의 대부분이 나왔군 !

In [28]:
for item in map_front.index:
    lat = map_front.loc[item, 'latitude']
    long = map_front.loc[item, 'longitude']
    folium.CircleMarker([lat, long], #위도 경도
        radius = map_front.loc[item,'ppl_sum']/100,
        popup = map_front.loc[item,'station_code'],
        color = "green",
        fill = True
    ).add_to(map_osm)
    
map_osm

- 상위 30개가 지도에 표시되었고, 원은 총 승하차 인원을 의미한다 !

In [29]:
for item in map_tail.index:
    lat = map_tail.loc[item, 'latitude']
    long = map_tail.loc[item, 'longitude']
    folium.CircleMarker([lat, long], #위도 경도
        radius = map_tail.loc[item,'ppl_sum']/100,
        popup = map_tail.loc[item,'station_code'],
        color = "green",
        fill = True
    ).add_to(map_osm)
    
map_osm

- 하위 30개의 버스정류장은 초록색으로 표시되었는데, 이는 서귀포시쪽에 있는 것을 지도를 통해 알 수 있다 !
- 원을 클릭하면 정류장 코드를 알 수 있다 !